In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from binance.client import Client
import ta
from sklearn.preprocessing import MinMaxScaler
import joblib
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

"""
Modelo de Predicción de Precios de Bitcoin con Indicadores Técnicos

Este modelo utiliza datos históricos de precios de Bitcoin (BTC) y varios indicadores técnicos para predecir los precios de cierre de Bitcoin en los siguientes 30 días. El modelo está diseñado para ayudar en la predicción de tendencias de precios a largo plazo.

Inputs:
    - X_train: Conjunto de datos de entrenamiento que contiene secuencias de datos históricos de 3 años (365 * 3 días) de precios de cierre de Bitcoin y valores de indicadores técnicos. La forma de X_train es (muestras, pasos de tiempo, características).
    - y_train: Conjunto de datos de entrenamiento que contiene los precios de cierre de Bitcoin para los siguientes 30 días después de cada secuencia de entrada. La forma de y_train es (muestras, 30).

Outputs:
    - model: El modelo entrenado que puede utilizarse para hacer predicciones de precios de cierre de Bitcoin en los siguientes 30 días.
    
Arquitectura del Modelo:
    - El modelo consta de dos capas LSTM (Long Short-Term Memory) para capturar relaciones temporales en los datos.
    - La primera capa LSTM tiene 100 unidades y tiene return_sequences=True para proporcionar secuencias intermedias.
    - La segunda capa LSTM tiene 50 unidades.
    - La capa de salida es una capa densa con 30 unidades, que produce 30 valores de precio de cierre para los siguientes 30 días.

Entrenamiento del Modelo:
    - El modelo se entrena durante 200 épocas utilizando el error cuadrático medio (MSE) como función de pérdida y el optimizador Adam.
    - Los datos de entrenamiento se dividen en conjuntos de entrenamiento y validación con una proporción del 80% y 20%, respectivamente.

Uso del Modelo:
    - Una vez entrenado, el modelo puede utilizarse para predecir los precios de cierre de Bitcoin en los siguientes 30 días a partir de una secuencia de datos históricos y valores de indicadores técnicos.

"""

# Autenticación en la API de Binance
api_key = 'your_api_key'
api_secret = 'your_api_secret'
client = Client(api_key, api_secret)

# Obtención de los datos históricos de precios de Bitcoin
klines = client.futures_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "10 years ago UTC")

# Creación de un dataframe con los datos
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                     'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 
                                     'taker_buy_quote_asset_volume', 'ignore'])

# Eliminación de las columnas que no se van a utilizar
data = data.drop(['timestamp', 'close_time', 'quote_asset_volume', 'number_of_trades', 
                  'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)

# Conversión de los precios a float
data['open'] = data['open'].astype(float)
data['high'] = data['high'].astype(float)
data['low'] = data['low'].astype(float)
data['close'] = data['close'].astype(float)
data['volume'] = data['volume'].astype(float)

# Cálculo de los indicadores técnicos
data['momentum'] = ta.momentum.roc(data['close'], fillna=True)
data['ATR'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'], fillna=True)
data['ADX'] = ta.trend.adx(data['high'], data['low'], data['close'], fillna=True)
data['MACD'] = ta.trend.macd_diff(data['close'], fillna=True)

# Crear y ajustar el scaler
scalers = {}
for column in ['open', 'high', 'low', 'close', 'momentum', 'ATR', 'ADX', 'MACD']:
    scaler = MinMaxScaler()
    data[column] = scaler.fit_transform(data[[column]])
    scalers[column] = scaler
    # Guardar el scaler para uso posterior
    joblib.dump(scaler, f'scalers/{column}_scaler.gz')

# Procesar los datos para la entrada del modelo
data['next_close'] = data['close'].shift(-30) # Precio de cierre 30 días después
data.dropna(inplace=True)

X = []
y = []

n = 365 * 3  # Número de pasos de tiempo en la secuencia: 3 años

# Añadir los indicadores técnicos a la entrada del modelo
for i in range(n, len(data)):
    X.append(data[['close', 'momentum', 'ATR', 'ADX', 'MACD']].iloc[i-n:i].values)
    y.append(data['next_close'].values[i:i+30])

# Convertir a numpy arrays
X, y = np.array(X), np.array(y)

# Redimensionar X a la forma adecuada para la entrada LSTM [muestras, pasos de tiempo, características]
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(30) # Se ajusta a la forma de la salida (30 días)
])

# Compilar el modelo
model.compile(loss='mse', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, batch_size=20, validation_data=(X_val, y_val))

# Guardar el modelo entrenado
model.save('modelos de entrenamiento/btc_price_prediction_model_Indicators_copy.h5')


Epoch 1/200
1295/1295 [==============================] - 27s 20ms/step - loss: 4.6816e-04 - val_loss: 9.4539e-05
Epoch 2/200
1295/1295 [==============================] - 26s 20ms/step - loss: 9.0963e-05 - val_loss: 5.9168e-05
Epoch 3/200
1295/1295 [==============================] - 26s 20ms/step - loss: 7.8568e-05 - val_loss: 5.1652e-05
Epoch 4/200
1295/1295 [==============================] - 26s 20ms/step - loss: 7.1777e-05 - val_loss: 4.7086e-05
Epoch 5/200
1295/1295 [==============================] - 26s 20ms/step - loss: 6.1554e-05 - val_loss: 4.4188e-05
Epoch 6/200
1295/1295 [==============================] - 26s 20ms/step - loss: 5.4670e-05 - val_loss: 5.1700e-05
Epoch 7/200
1295/1295 [==============================] - 26s 20ms/step - loss: 5.0675e-05 - val_loss: 6.7797e-05
Epoch 8/200
1295/1295 [==============================] - 26s 20ms/step - loss: 5.3247e-05 - val_loss: 5.8937e-05
Epoch 9/200
1295/1295 [==============================] - 26s 20ms/step - loss: 5.1652e-05 - val_